# Adding New York bus routes to OSMnx map

## Section 1: Extracting bus stops from KML
KML source: 
<a href="https://www.google.com/maps/d/u/0/viewer?mid=1Y-euNeFcsu06Zxfdl6u6-sca3Yp-KYY&ll=40.75502933824687%2C-74.00066380083778&z=18">
MTA Bus Map
</a>

### Opening KML file and extracting placemarks

In [1]:
from fastkml import KML
from fastkml import Placemark, Point, StyleUrl, Style
from fastkml.utils import find, find_all
import os

/Users/charlierothschild/miniforge3/envs/DSA/lib/python3.13/site-packages/fastkml/config.py:39: UserWarning: Package `lxml` missing. Pretty print will be disabled
  warnings.warn("Package `lxml` missing. Pretty print will be disabled")  # noqa: B028


In [2]:
#read .kml file as KML object with fastkml
k = KML.parse("../data/doc.kml")

In [3]:
#extract all placemarks in .kml file
placemarks = list(find_all(k, of_type=Placemark))

In [4]:
#print the number of placemarks
print(len(placemarks))

15042


In [5]:
print(placemarks[0].description)

name: HYLAN BLVD/MIDLAND AV<br>routes: S51, S78, S79+, S81<br>direction: SW<br>link: https://bustime.mta.info/m/index?q=200237<br>id: MTA_200237<br>Latitude,Longitude: 40.577699, -74.102611


### Parsing Placemark Description

In [6]:
#extravcting info from placemarker description

def parse_placemark_info(desc_str):
    """
    Parse a description string like:
      "name: HYLAN BLVD/MIDLAND AV<br>"
      "routes: S51, S78, S79+, S81<br>"
      "direction: SW<br>"
      "link: https://…<br>"
      "id: MTA_200237<br>"
      "Latitude,Longitude: 40.577699, -74.102611"
    into a dict with proper types.
    """
    # 1) split into lines
    parts = desc_str.split('<br>')
    
    data = {}
    for part in parts:
        if not part.strip():
            continue
        # split on the first ": "
        key, val = part.split(': ', 1)
        data[key.strip()] = val.strip()
    
    # 2) post‑process some fields:
    #   routes → list of route codes
    if 'routes' in data:
        data['routes'] = [r.strip() for r in data['routes'].split(',')]
    
    #   Latitude,Longitude → two floats
    if 'Latitude,Longitude' in data:
        lat_str, lon_str = data['Latitude,Longitude'].split(',', 1)
        data['latitude']  = float(lat_str)
        data['longitude'] = float(lon_str)
        # optional: you can delete the original key
        del data['Latitude,Longitude']
    
    return data

In [7]:
# testing
test_placemark_info = parse_placemark_info(placemarks[0].description)
test_placemark_info

{'name': 'HYLAN BLVD/MIDLAND AV',
 'routes': ['S51', 'S78', 'S79+', 'S81'],
 'direction': 'SW',
 'link': 'https://bustime.mta.info/m/index?q=200237',
 'id': 'MTA_200237',
 'latitude': 40.577699,
 'longitude': -74.102611}

placemark.description seems to have all the data we might need for the OSM

## Section 2: Getting a Placemark based on OSMnx bus route name

In [8]:
import multiprocessing as mp
import numpy as np
import osmnx as ox
from matplotlib import pyplot as plt
import networkx as nx

ox.__version__

'2.0.2'

### Converting new york digraph to GeodataFrame

In [9]:
# open New York
place = "New York, New York"
G = ox.graph.graph_from_place(place, network_type="drive")
Gp = ox.projection.project_graph(G)

Getting bus stops

Should I use openstreetmap's data or just add my info to each node by using kd tree? <br>
I'll use a KD tree because this is a DSA project <br>

In [10]:
# convert drive multidigraph nodes to geodataframe
gdf_nodes = ox.convert.graph_to_gdfs(
    G, nodes=True, edges=False, node_geometry=True,
    fill_edge_geometry=False)

In [11]:
# # display it on map
# gdf_nodes.explore()

The drive network loads much faster than the entire open street map network

### Locating the node closest to a placemarker

In [12]:
def findNearestNode2Placemark(G, placemark_info):
    '''
    get the ID of the node nearest to a placemark
    '''
    placemark_longitude, placemark_latitude = placemark_info['longitude'], placemark_info['latitude']

    nearest_node = ox.distance.nearest_nodes(G,
                                         placemark_longitude, placemark_latitude,
                                         return_dist=True)
    
    return nearest_node

In [13]:
#finding node nearest to placemark[0]
nearest_node = findNearestNode2Placemark(G, test_placemark_info)

In [14]:
# fetch the node geometry based on node id
nearest_node_id = nearest_node[0]
gdf_nodes.loc[nearest_node_id]

y                                    40.577717
x                                    -74.10254
highway                                    NaN
ref                                        NaN
street_count                                 2
junction                                   NaN
railway                                    NaN
geometry        POINT (-74.1025398 40.5777173)
Name: 5490128948, dtype: object

In [15]:
# visualizing node nearest to placemarker
import geopandas as gpd
from shapely import (
    Point, LineString)

# create a new geodataframe with the nearest node and new point
nearest_node_dict = {'col1': ['Bus Station', 'Nearest Node'],
                     'geometry': [Point(test_placemark_info['longitude'], test_placemark_info['latitude']),
                                        LineString([
                                      Point(gdf_nodes.loc[nearest_node_id].x,
                                            gdf_nodes.loc[nearest_node_id].y),
                                      Point(test_placemark_info['longitude'], test_placemark_info['latitude'])])]}
# convert dictionary to geodataframe
nearest_node_gdf = gpd.GeoDataFrame(nearest_node_dict, crs="EPSG:4326")
# nearest node map reference
nearest_node_map = nearest_node_gdf.explore(color="red")
# # combine nearest node with existing node map
# gdf_nodes.explore(m=nearest_node_map)

### adding placemark info to gdf node

In [16]:
def addPlacemark2GDF(G, node_id, placemark_info):
    print(placemark_info)
    attributes = {node_id : placemark_info}
    nx.set_node_attributes(G, attributes)
    
    return G.nodes[node_id]

In [17]:
info = addPlacemark2GDF(G, nearest_node_id, test_placemark_info)
print(info)


{'name': 'HYLAN BLVD/MIDLAND AV', 'routes': ['S51', 'S78', 'S79+', 'S81'], 'direction': 'SW', 'link': 'https://bustime.mta.info/m/index?q=200237', 'id': 'MTA_200237', 'latitude': 40.577699, 'longitude': -74.102611}
{'y': 40.5777173, 'x': -74.1025398, 'street_count': 2, 'name': 'HYLAN BLVD/MIDLAND AV', 'routes': ['S51', 'S78', 'S79+', 'S81'], 'direction': 'SW', 'link': 'https://bustime.mta.info/m/index?q=200237', 'id': 'MTA_200237', 'latitude': 40.577699, 'longitude': -74.102611}


In [18]:
import time
from tqdm import tqdm

In [19]:

#add descriptions to closest neighbor (warning: some bus stations may appear on the same node. Be sure to handle properly)
def get_attribute_list(G, placemarks):
    # extracts description dicts from a list of placemarks
    allDescriptions = list(map(lambda x: parse_placemark_info(x.description), placemarks))
    print(f'number of placemarks: {len(allDescriptions)}')


    #create a set containg each nearest node id
    start = time.time()
    nearest_node_ids = []

    #display a progress bar
    for i in tqdm(range(len(allDescriptions)), desc="finding neighbors"):
        nearest_node_ids.append(findNearestNode2Placemark(G, allDescriptions[i])[0])
    end = time.time()

    print(f'number of nodes: {len(set(nearest_node_ids))}')

    
    return list(zip(nearest_node_ids, allDescriptions))


In [20]:
#this code takes 20+ minutes to run
attribute_list = get_attribute_list(G, placemarks)
#ended up with one fewer node than descriptions... i think we're fine

number of placemarks: 15042


finding neighbors: 100%|██████████| 15042/15042 [20:42<00:00, 12.11it/s]  

number of nodes: 10259


In [21]:
#adding bus stations to each node
def add_path_attributes(G, attribute_list):
    #Giving each node an empty list as its bus values
    nx.set_node_attributes(G, [], 'bus_stops')
    for attributeTup in attribute_list:
        #add the bus id to a list of bus stops
        G.nodes[attributeTup[0]]['bus_stops'] = G.nodes[attributeTup[0]]['bus_stops'] + [attributeTup[1]['id']]

        #store the attribute dictionary the bus id 
        G.nodes[attributeTup[0]][attributeTup[1]['id']] = [attributeTup[1]]

In [34]:
#add bus stop attributes to each node with a bus stop
add_path_attributes(G, attribute_list)

In [23]:
G.nodes[list(attribute_list)[0][0]]

{'y': 40.5777173,
 'x': -74.1025398,
 'street_count': 2,
 'name': 'HYLAN BLVD/MIDLAND AV',
 'routes': ['S51', 'S78', 'S79+', 'S81'],
 'direction': 'SW',
 'link': 'https://bustime.mta.info/m/index?q=200237',
 'id': 'MTA_200237',
 'latitude': 40.577699,
 'longitude': -74.102611,
 'bus_stops': ['MTA_200237'],
 'MTA_200237': [{'name': 'HYLAN BLVD/MIDLAND AV',
   'routes': ['S51', 'S78', 'S79+', 'S81'],
   'direction': 'SW',
   'link': 'https://bustime.mta.info/m/index?q=200237',
   'id': 'MTA_200237',
   'latitude': 40.577699,
   'longitude': -74.102611}]}

In [24]:
# convert drive multidigraph nodes to geodataframe
gdf_nodes = ox.convert.graph_to_gdfs(
    G, nodes=True, edges=False, node_geometry=True,
    fill_edge_geometry=False)

In [25]:
# gdf_nodes.explore(m=nearest_node_map)

In [26]:
def stringer(data):
    if isinstance(data, (int, float)):
        return data
    elif isinstance(data, dict):
        return {k: stringer(v) for k, v in data.items()}
    elif data is None:
        return "None"
    elif isinstance(data, (list, tuple)):
         return [stringer(item) for item in data]
    else:
        return str(data)


In [39]:
#saving as gml
filepath = "./graph_data/bus_stops.graphml"
ox.io.save_graphml(G, filepath)
G = ox.io.load_graphml(filepath)

In [30]:
from pathlib import Path

In [31]:
Path("data").mkdir(parents=True, exist_ok=True)

In [ ]:
# get all "amenities" and save as a geopackage via geopandas
gdf = ox.features.features_from_place(place, tags={"amenity": True})
gdf = gdf.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)
gdf.to_file("./data/pois.gpkg", driver="GPKG")

In [41]:
# open New York
place = "New York, New York"
G = ox.graph.graph_from_place(place, network_type="drive")
Gp = ox.projection.project_graph(G)

In [108]:
NY_gdf_nodes, NY_gdf_edges = ox.convert.graph_to_gdfs(G)

In [109]:
NY_gdf_nodes.head
    

<bound method NDFrame.head of                      y          x            highway  ref  street_count  \
osmid                                                                     
39076461     40.786345 -73.794748  motorway_junction   33             3   
39076490     40.762429 -73.757091  motorway_junction  31W             3   
39076504     40.753467 -73.744164  motorway_junction  30W             3   
42421728     40.798048 -73.960044    traffic_signals  NaN             3   
42421731     40.798654 -73.961474    traffic_signals  NaN             4   
...                ...        ...                ...  ...           ...   
12798385432  40.857737 -73.899772                NaN  NaN             3   
12798385434  40.858040 -73.899380                NaN  NaN             3   
12798385435  40.858334 -73.899418                NaN  NaN             3   
12803554307  40.659668 -73.987593                NaN  NaN             3   
12804565238  40.589305 -74.093242     turning_circle  NaN             

In [110]:
from collections import defaultdict
import pandas as pd
import geopandas as gpd

In [111]:
#adding bus stations to each node
def add_attributes_to_gdf(gdf, attribute_list):
    bus_node_dict = {}
    bus_info_dict = {}

    for attributretup in attribute_list:
        bus_info_dict[attributretup[1]['id']] = attributretup[1]

        if attributretup[0] in bus_node_dict:
            bus_node_dict[attributretup[0]].append(attributretup[1]['id'])
            continue

        bus_node_dict[attributretup[0]] = [attributretup[1]['id']]


    osmids = list(bus_node_dict.keys())
    stops = list(bus_node_dict.values())

    df_dict = {'osmid': osmids, 'stops': stops}
    
    return df_dict, bus_info_dict

In [112]:
attr = add_attributes_to_gdf(NY_gdf_nodes, attribute_list)
df = pd.DataFrame(attr[0])
df

,osmid,stops
0,5490128948,[MTA_200237]
1,42965730,"[MTA_200285, MTA_200153, MTA_200178, MTA_90500..."
2,42955743,[MTA_200292]
3,2663757129,[MTA_200367]
4,2663757126,[MTA_200479]
...,...,...
10254,42991585,[MTA_250050]
10255,42991502,[MTA_250051]
10256,42991570,[MTA_250068]
10257,359875121,[MTA_805079]


In [113]:
#adding bus ids to all nodes associated with bus stations
bus_gdf = NY_gdf_nodes.merge(df, on='osmid',how='left')

In [114]:
import pyogrio
pyogrio.list_drivers() 

{'PCIDSK': 'rw',
 'PDS4': 'rw',
 'VICAR': 'rw',
 'MBTiles': 'rw',
 'EEDA': 'r',
 'OGCAPI': 'r',
 'ESRI Shapefile': 'rw',
 'MapInfo File': 'rw',
 'UK .NTF': 'r',
 'LVBAG': 'r',
 'OGR_SDTS': 'r',
 'S57': 'rw',
 'DGN': 'rw',
 'OGR_VRT': 'r',
 'Memory': 'rw',
 'CSV': 'rw',
 'NAS': 'r',
 'GML': 'rw',
 'GPX': 'rw',
 'LIBKML': 'rw',
 'KML': 'rw',
 'GeoJSON': 'rw',
 'GeoJSONSeq': 'rw',
 'ESRIJSON': 'r',
 'TopoJSON': 'r',
 'Interlis 1': 'rw',
 'Interlis 2': 'rw',
 'OGR_GMT': 'rw',
 'GPKG': 'rw',
 'SQLite': 'rw',
 'WAsP': 'rw',
 'OpenFileGDB': 'rw',
 'DXF': 'rw',
 'CAD': 'r',
 'FlatGeobuf': 'rw',
 'Geoconcept': 'rw',
 'GeoRSS': 'rw',
 'VFK': 'r',
 'PGDUMP': 'rw',
 'OSM': 'r',
 'GPSBabel': 'rw',
 'OGR_PDS': 'r',
 'WFS': 'r',
 'OAPIF': 'r',
 'EDIGEO': 'r',
 'SVG': 'r',
 'Idrisi': 'r',
 'ODS': 'rw',
 'XLSX': 'rw',
 'Elasticsearch': 'rw',
 'Carto': 'rw',
 'AmigoCloud': 'rw',
 'SXF': 'r',
 'Selafin': 'rw',
 'JML': 'rw',
 'PLSCENES': 'r',
 'CSW': 'r',
 'VDV': 'rw',
 'GMLAS': 'r',
 'MVT': 'rw',
 'NGW':

In [121]:
#saving nodes to file
bus_gdf.to_file('./graph_data/gpkg/NY_gdf_nodes.gpkg')

In [118]:
import json

In [117]:
#saving edges to file
NY_gdf_edges.to_file('./graph_data/gpkg/NY_gdf_edges.gpkg')

In [ ]:
#saving bus stop dict to file
filename = "graph_data/gpkg/bus_info_dict.json"
with open(filename, 'w') as file:
    json.dump(attr[1], file, indent=4)

print(f"Dictionary saved to {filename}")

Dictionary saved to bus_info_dict.json
